In [ ]:
import pandas as pd
import numpy as np
import json
import requests
from pathlib import Path

pd.set_option('display.max_columns', 300)

In [ ]:
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Ipswich Town', 'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd', 'Nottingham Forest', 'Southampton', 'Tottenham', 'West Ham', 'Wolves']

In [ ]:
# fetch FPL data online
fpl_online_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_online_df = pd.DataFrame(fpl_online_data['elements'])
fpl_online_df['team_name'] = [teams[i] for i in fpl_online_df['team']-1]
fpl_online_df['name'] = fpl_online_df.apply(lambda x: x['first_name'] + ' ' + x['second_name'], axis=1)
fpl_online_df

In [ ]:
# old data
my_path = Path('../data/modeling/fpl_df.csv')
fpl_df = pd.read_csv(my_path, index_col=0)
fpl_df 

In [ ]:
fpl_df_latest = fpl_df.groupby(['first_name', 'second_name']).last().reset_index()
fpl_df_latest['name'] = fpl_df_latest.apply(lambda x: x['first_name'] + ' ' + x['second_name'], axis=1)
fpl_df_latest

In [ ]:
# only keep players that are found this season
fpl_df_latest = fpl_df_latest[fpl_df_latest['name'].isin(fpl_online_df.name.unique())].reset_index()
fpl_df_latest

In [ ]:
correct_teams = []
correct_positions = []
for ix, row in fpl_df_latest.iterrows():
    my_name = row['name']
    my_position = row.element_type
    online_team = fpl_online_df.loc[fpl_online_df.name==my_name, 'team'].item()
    online_position = fpl_online_df.loc[fpl_online_df.name==my_name, 'element_type'].item()
    correct_teams.append(online_team)
    correct_positions.append(online_position)

fpl_df_latest['team'] = correct_teams
fpl_df_latest['element_type'] = correct_positions

fpl_df_latest

In [ ]:
fpl_df_latest['team_name'] = [teams[i] for i in fpl_df_latest['team']-1]
fpl_df_latest